# 기상청 빅데이터 콘테스트

## 날씨에 따른 소비 예측


### 1. 데이터 전처리


#### 1) 대기환경 데이터 전처리

18 ~ 19년도 대기환경(오존, 미세먼지, 초미세먼지) 데이터 통합

In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/weather_bigdata_contest/code/

/content/drive/MyDrive/weather_bigdata_contest/code




에어코리아에서 받은 18~19년 대기환경 데이터


In [ ]:
air_df = pd.read_csv('../data/air_data.csv', encoding = 'CP949')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
air_df.head()

,지역,망,측정소코드,측정소명,측정일시,SO2,CO,O3,NO2,PM10,PM25,주소
0,서울 중구,도시대기,111121,중구,2019010101,0.003,0.7,0.003,0.054,39.0,24.0,서울 중구 덕수궁길 15
1,서울 중구,도시대기,111121,중구,2019010102,0.003,0.8,0.002,0.056,38.0,27.0,서울 중구 덕수궁길 15
2,서울 중구,도시대기,111121,중구,2019010103,0.003,0.9,0.002,0.057,42.0,28.0,서울 중구 덕수궁길 15
3,서울 중구,도시대기,111121,중구,2019010104,0.003,0.8,0.002,0.054,42.0,31.0,서울 중구 덕수궁길 15
4,서울 중구,도시대기,111121,중구,2019010105,0.003,0.8,0.002,0.048,49.0,33.0,서울 중구 덕수궁길 15


대한민국 인구수 8위: 서울, 부산, 인천, 대구, 대전, 광주, 울산, 수원만 추출

* 먼저, 서울, 부산, 인천, 대구, 대전, 광주, 울산 추출
* 경기도에 속해있는 수원 추출
* 2가지 병합

In [ ]:
def air_mean(df):
    df['시도'] = df.지역.map(lambda x: x.split()[0])
    df['측정일'] = df.측정일시 // 100
    return df[['시도','측정일','O3','PM10','PM25']].groupby(['시도','측정일']).mean()

In [ ]:
air_mean_df = air_mean(air_df)

In [ ]:
air_mean_df = air_mean_df.reset_index()

In [ ]:
air_df['시군구']=air_df.지역.map(lambda x:x.split()[1])

In [ ]:
condition = (air_mean_df.시도 == '서울')|(air_mean_df.시도 == '부산')|(air_mean_df.시도 == '인천')|(air_mean_df.시도 == '대구')|(air_mean_df.시도 == '대전')|(air_mean_df.시도 == '광주')|(air_mean_df.시도 == '울산')

air_mean_df[condition]

,시도,측정일,O3,PM10,PM25
2920,광주,20180101,0.020509,50.337963,18.763889
2921,광주,20180102,0.010901,71.245192,39.188406
2922,광주,20180103,0.021578,29.644550,14.437811
2923,광주,20180104,0.018392,33.753488,18.446512
2924,광주,20180105,0.017606,40.827907,26.046512
...,...,...,...,...,...
8755,인천,20191227,0.023184,31.929467,19.973354
8756,인천,20191228,0.012289,41.895062,21.630841
8757,인천,20191229,0.010069,45.524691,25.029321
8758,인천,20191230,0.018284,33.023112,20.793651


In [ ]:
suwon_df = air_df[air_df.시군구=='수원시'][['시도','시군구','측정일','O3','PM10','PM25']].groupby(['시도','시군구','측정일']).mean().reset_index()

In [ ]:
suwon_df.시도='수원'
del suwon_df['시군구']

In [ ]:
air_top8_df = pd.concat([suwon_df,air_mean_df[condition]], axis=0)

In [ ]:
air_top8_df.columns = ['지역', '날짜', '오존(O3)', '미세먼지(PM10)', '초미세먼지(PM2.5)']

In [ ]:
air_top8_df.날짜 = air_top8_df.날짜.astype('str').map(lambda x: x[:4]+'-'+x[4:6]+'-'+x[6:])

In [ ]:
air_top8_df.날짜 = pd.to_datetime(air_top8_df.날짜)

In [ ]:
air_top8_df.to_csv('../data/air_top8.csv', index = False, encoding = 'CP949')

#### 2) 기상데이터 전처리

In [ ]:
weather_top8_df = pd.read_csv('../data/weather_top8.csv', encoding = 'CP949')
weather_top8_df.head()

,지점,지점명,일시,평균기온(°C),최저기온(°C),최고기온(°C),일강수량(mm),평균 풍속(m/s),평균 상대습도(%),평균 전운량(1/10)
0,108,서울,2018-01-01,-1.3,-5.1,3.8,NaN,1.4,39.1,1.0
1,108,서울,2018-01-02,-1.8,-4.3,1.8,NaN,1.8,42.0,2.3
2,108,서울,2018-01-03,-4.7,-7.1,-0.4,NaN,2.2,42.3,3.6
3,108,서울,2018-01-04,-4.7,-8.7,-0.7,NaN,1.4,43.0,6.5
4,108,서울,2018-01-05,-3.0,-5.6,1.6,NaN,1.7,48.4,3.6


In [ ]:
# 지점 칼럼 없애기
weather_top8_df.drop(['지점'], inplace = True, axis = 1)

# 컬럼명 바꾸기
weather_top8_df = weather_top8_df.rename({'지점명' : '지역', '일시' : '날짜'}, axis='columns')
weather_top8_df.날짜 = pd.to_datetime(weather_top8_df.날짜)

weather_top8_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5840 entries, 0 to 5839
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   지역            5840 non-null   object        
 1   날짜            5840 non-null   datetime64[ns]
 2   평균기온(°C)      5840 non-null   float64       
 3   최저기온(°C)      5840 non-null   float64       
 4   최고기온(°C)      5840 non-null   float64       
 5   일강수량(mm)      2079 non-null   float64       
 6   평균 풍속(m/s)    5835 non-null   float64       
 7   평균 상대습도(%)    5840 non-null   float64       
 8   평균 전운량(1/10)  5840 non-null   float64       
dtypes: datetime64[ns](1), float64(7), object(1)
memory usage: 410.8+ KB


풍속 결측치는 근처 지역의 평균풍속 이용


*   인천 -> 파주
*   대전 -> 금산
*   대구 -> 영천
*   울산 -> 부산


In [ ]:
weather_top8_df[weather_top8_df['평균 풍속(m/s)'].isna()]

,지역,날짜,평균기온(°C),최저기온(°C),최고기온(°C),일강수량(mm),평균 풍속(m/s),평균 상대습도(%),평균 전운량(1/10)
940,인천,2018-07-30,31.1,25.8,35.6,NaN,NaN,57.3,3.5
2685,대전,2019-05-11,20.1,11.3,28.9,NaN,NaN,45.8,1.0
2880,대전,2019-11-22,9.9,3.0,18.1,NaN,NaN,62.8,1.1
3390,대구,2019-04-16,17.0,7.3,25.3,NaN,NaN,26.1,5.5
4177,울산,2019-06-12,18.1,15.3,21.8,NaN,NaN,78.9,3.3


In [ ]:
# 인천은 파주, 대전은 금산, 대구는 영천, 울산은 부산으로 결측치 대체
weather_top8_df.loc[940, '평균 풍속(m/s)'] = 2.2
weather_top8_df.loc[2685, '평균 풍속(m/s)'] = 1.1
weather_top8_df.loc[2880, '평균 풍속(m/s)'] = 0.7
weather_top8_df.loc[3390, '평균 풍속(m/s)'] = 1.4
weather_top8_df.loc[4177, '평균 풍속(m/s)'] = 2.4

In [ ]:
weather_top8_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5840 entries, 0 to 5839
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   지역            5840 non-null   object        
 1   날짜            5840 non-null   datetime64[ns]
 2   평균기온(°C)      5840 non-null   float64       
 3   최저기온(°C)      5840 non-null   float64       
 4   최고기온(°C)      5840 non-null   float64       
 5   일강수량(mm)      2079 non-null   float64       
 6   평균 풍속(m/s)    5840 non-null   float64       
 7   평균 상대습도(%)    5840 non-null   float64       
 8   평균 전운량(1/10)  5840 non-null   float64       
dtypes: datetime64[ns](1), float64(7), object(1)
memory usage: 410.8+ KB


일강수량 기준


*   눈, 비 오지 않음 : NaN
*   0.1mm 미만의 눈, 비 : 0

따라서, NaN을 0으로 처리


In [ ]:
weather_top8_df['일강수량(mm)'].fillna(0, inplace = True)
weather_top8_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5840 entries, 0 to 5839
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   지역            5840 non-null   object        
 1   날짜            5840 non-null   datetime64[ns]
 2   평균기온(°C)      5840 non-null   float64       
 3   최저기온(°C)      5840 non-null   float64       
 4   최고기온(°C)      5840 non-null   float64       
 5   일강수량(mm)      5840 non-null   float64       
 6   평균 풍속(m/s)    5840 non-null   float64       
 7   평균 상대습도(%)    5840 non-null   float64       
 8   평균 전운량(1/10)  5840 non-null   float64       
dtypes: datetime64[ns](1), float64(7), object(1)
memory usage: 410.8+ KB


#### 3) 기상데이터와 대기환경데이터 합치고 인구비율에 따라 가중치 두고 통합

In [ ]:
air_top8_df['날짜'] = pd.to_datetime(air_top8_df['날짜'])
air_top8_df = air_top8_df.sort_values('날짜')
weather_top8_df['날짜'] = pd.to_datetime(weather_top8_df['날짜'])
weather_top8_df = weather_top8_df.sort_values('날짜')

In [ ]:
weather_df = pd.merge_asof(weather_top8_df, air_top8_df, on = '날짜', by = '지역')
weather_df.head()

,지역,날짜,평균기온(°C),최저기온(°C),최고기온(°C),일강수량(mm),평균 풍속(m/s),평균 상대습도(%),평균 전운량(1/10),오존(O3),미세먼지(PM10),초미세먼지(PM2.5)
0,서울,2018-01-01,-1.3,-5.1,3.8,0.0,1.4,39.1,1.0,0.010054,44.587321,21.150087
1,대전,2018-01-01,-0.7,-5.3,4.9,0.0,1.0,60.0,0.5,0.012733,53.368201,22.658333
2,인천,2018-01-01,-0.3,-2.7,2.7,0.0,1.6,45.9,1.8,0.015464,37.430556,18.641758
3,대구,2018-01-01,1.3,-1.3,5.0,0.0,3.1,36.1,0.0,0.020099,43.275641,20.051613
4,울산,2018-01-01,2.1,-0.4,6.2,0.0,3.3,32.0,0.0,0.027346,37.467262,15.433735


In [ ]:
weather_df['일교차(°C)'] = weather_df['최고기온(°C)'] - weather_df['최저기온(°C)']
weather_df.drop(['최고기온(°C)', '최저기온(°C)', '초미세먼지(PM2.5)'], axis = 1, inplace = True)

In [ ]:
weather_df = weather_df[['지역', '날짜', '평균기온(°C)',  '일교차(°C)', '일강수량(mm)', '평균 풍속(m/s)', '평균 상대습도(%)',
       '평균 전운량(1/10)', '오존(O3)', '미세먼지(PM10)']]

In [ ]:
weather_df.head()

,지역,날짜,평균기온(°C),일교차(°C),일강수량(mm),평균 풍속(m/s),평균 상대습도(%),평균 전운량(1/10),오존(O3),미세먼지(PM10)
0,서울,2018-01-01,-1.3,8.9,0.0,1.4,39.1,1.0,0.010054,44.587321
1,대전,2018-01-01,-0.7,10.2,0.0,1.0,60.0,0.5,0.012733,53.368201
2,인천,2018-01-01,-0.3,5.4,0.0,1.6,45.9,1.8,0.015464,37.430556
3,대구,2018-01-01,1.3,6.3,0.0,3.1,36.1,0.0,0.020099,43.275641
4,울산,2018-01-01,2.1,6.6,0.0,3.3,32.0,0.0,0.027346,37.467262


인구비율에 대한 가중치  
: 각 도시의 인구수 / top 8 도시의 총 인구수

(예) 평균 기온

(A 도시 가중치 * A 도시 평균 기온) + (B 도시 가중치 * B 도시 평균 기온) = 전국 평균 기온

In [ ]:
population_ratio = pd.read_csv('../data/population_ratio.csv', encoding='CP949')
population_ratio.columns = ['지역', '총 인구수', '가중치']
population_ratio.head(10)

,지역,총 인구수,가중치
0,수원,1197815.5,0.050179
1,광주,1457902.0,0.061075
2,대구,2449900.0,0.102632
3,대전,1482403.0,0.062101
4,부산,3427647.0,0.143592
5,서울,9747365.0,0.408340
6,울산,1151821.0,0.048253
7,인천,2955834.0,0.123827


In [ ]:
weather_list = []
for i in population_ratio['지역']:
    weather_list.append(weather_df[weather_df['지역'] == i].iloc[:, 2:].values * population_ratio['가중치'][population_ratio['지역'] == i].values)
weather_array = np.array(weather_list)

In [ ]:
weather_total_df = pd.DataFrame(weather_array.sum(axis =0))

In [ ]:
weather_total_df.shape

(730, 8)

In [ ]:
weather_total_df.columns = ['평균기온(°C)',  '일교차(°C)', '일강수량(mm)', '평균 풍속(m/s)', '평균 상대습도(%)',
       '평균 전운량(1/10)', '오존(O3)', '미세먼지(PM10)']
weather_total_df.head()

,평균기온(°C),일교차(°C),일강수량(mm),평균 풍속(m/s),평균 상대습도(%),평균 전운량(1/10),오존(O3),미세먼지(PM10)
0,0.066157,8.198700,0.000000,1.844893,41.267581,0.731871,0.015544,44.032069
1,-0.000201,7.910404,0.000000,2.003189,47.229263,2.787500,0.013179,53.742579
2,-2.328937,6.682618,0.000000,2.512411,39.916465,3.425790,0.017566,34.601134
3,-2.419582,7.470117,0.000000,1.780529,44.077518,6.876999,0.011983,42.672343
4,-0.734189,7.276539,0.114874,1.984833,50.195533,3.891405,0.011894,51.988526


In [ ]:
weather_total_df['날짜'] = weather_df[weather_df['지역'] == '서울']['날짜'].values
weather_total_df = weather_total_df[['날짜', '평균기온(°C)',  '일교차(°C)', '일강수량(mm)', '평균 풍속(m/s)', '평균 상대습도(%)',
       '평균 전운량(1/10)', '오존(O3)', '미세먼지(PM10)']]
weather_total_df.head()

,날짜,평균기온(°C),일교차(°C),일강수량(mm),평균 풍속(m/s),평균 상대습도(%),평균 전운량(1/10),오존(O3),미세먼지(PM10)
0,2018-01-01,0.066157,8.198700,0.000000,1.844893,41.267581,0.731871,0.015544,44.032069
1,2018-01-02,-0.000201,7.910404,0.000000,2.003189,47.229263,2.787500,0.013179,53.742579
2,2018-01-03,-2.328937,6.682618,0.000000,2.512411,39.916465,3.425790,0.017566,34.601134
3,2018-01-04,-2.419582,7.470117,0.000000,1.780529,44.077518,6.876999,0.011983,42.672343
4,2018-01-05,-0.734189,7.276539,0.114874,1.984833,50.195533,3.891405,0.011894,51.988526


In [ ]:
weather_total_df.to_csv('../data/weather_total.csv', encoding = 'CP949', index = False)

#### 4) 판매 데이터 전처리

In [4]:
buy18_1 = pd.read_csv('../data/buy2018_1.csv')
buy18_2 = pd.read_csv('../data/buy2018_2.csv')
buy19_1 = pd.read_csv('../data/buy2019_1.csv')
buy19_2 = pd.read_csv('../data/buy2019_2.csv')

In [5]:
buy18_1.rename(columns={'buy2018_1.date':'날짜', 'buy2018_1.sex':'성별',
                        'buy2018_1.age':'연령', 'buy2018_1.big_cat':'대분류',
                       'buy2018_1.sm_cat':'소분류','buy2018_1.qty':'판매량'}, inplace=True)
buy18_2.rename(columns={'buy2018_2.date':'날짜', 'buy2018_2.sex':'성별',
                        'buy2018_2.age':'연령', 'buy2018_2.big_cat':'대분류',
                       'buy2018_2.sm_cat':'소분류','buy2018_2.qty':'판매량'}, inplace=True)
buy19_1.rename(columns={'buy2019_1.date':'날짜', 'buy2019_1.sex':'성별',
                        'buy2019_1.age':'연령', 'buy2019_1.big_cat':'대분류',
                       'buy2019_1.sm_cat':'소분류','buy2019_1.qty':'판매량'}, inplace=True)
buy19_2.rename(columns={'buy2019_2.date':'날짜', 'buy2019_2.sex':'성별',
                        'buy2019_2.age':'연령', 'buy2019_2.big_cat':'대분류',
                       'buy2019_2.sm_cat':'소분류','buy2019_2.qty':'판매량'}, inplace=True)

buy18_1.drop('Unnamed: 0', axis=1, inplace=True)
buy18_2.drop('Unnamed: 0', axis=1, inplace=True)
buy19_1.drop('Unnamed: 0', axis=1, inplace=True)
buy19_2.drop('Unnamed: 0', axis=1, inplace=True)

buy_df = pd.concat([buy18_1, buy18_2, buy19_1, buy19_2], axis=0)
buy_df.to_csv('../data/buy.csv', encoding = 'CP949', index = False)

In [6]:
buy_df.head()

,날짜,성별,연령,대분류,소분류,판매량
0,20180101,F,20,식품,가공란,37
1,20180101,F,30,식품,가공란,16
2,20180101,F,40,식품,가공란,9
3,20180101,F,50,식품,가공란,3
4,20180101,M,20,식품,가공란,13


In [7]:
buy_pivot = pd.pivot_table(data=buy_df, index=['날짜', '대분류','소분류'],
                           columns=['연령','성별'],values = '판매량',
                           aggfunc='sum')

In [8]:
buy_pivot = buy_pivot.reset_index()
buy_pivot

연령            날짜    대분류      소분류     20         ...     40     50          60      
성별                                    F      M  ...      M      F     M     F     M
0       20180101  냉난방가전  가열식 가습기    1.0    1.0  ...    2.0    NaN   NaN   NaN   NaN
1       20180101  냉난방가전  공기정화 용품   10.0    2.0  ...   24.0    2.0  12.0   2.0   1.0
2       20180101  냉난방가전    공기청정기    1.0    2.0  ...   18.0    6.0   4.0   2.0   3.0
3       20180101  냉난방가전      냉풍기    NaN    NaN  ...    NaN    NaN   NaN   NaN   NaN
4       20180101  냉난방가전      돈풍기    NaN    NaN  ...    1.0    NaN   NaN   NaN   NaN
...          ...    ...      ...    ...    ...  ...    ...    ...   ...   ...   ...
267702  20191231     식품       홍어    NaN    NaN  ...    1.0    1.0   NaN   1.0   NaN
267703  20191231     식품       홍차   10.0    2.0  ...    9.0    1.0   2.0   NaN   1.0
267704  20191231     식품      환자식    1.0    NaN  ...    6.0   12.0   6.0   6.0   1.0
267705  20191231     식품        회  497.0  217.0  ...  206.0  105.0  56.0  42.0  16.0
267706  20191231     식품      흰우유   66.0   37.0  ...   77.0   49.0  24.0  11.0  14.0

[267707 rows x 13 columns]

In [9]:
buy_pivot.columns = ['날짜','대분류','소분류',
                    '20대 여성 판매량(개)','20대 남성 판매량(개)',
                    '30대 여성 판매량(개)','30대 남성 판매량(개)',
                    '40대 여성 판매량(개)','40대 남성 판매량(개)',
                    '50대 여성 판매량(개)','50대 남성 판매량(개)',
                    '60대 여성 판매량(개)','60대 남성 판매량(개)']

buy_pivot = buy_pivot.fillna(0)

buy_pivot['일별 판매 합계량(개)']=buy_pivot.loc[:,'20대 여성 판매량(개)':'60대 남성 판매량(개)'].sum(axis=1)

buy_pivot.날짜 = buy_pivot.날짜.astype('str').map(lambda x:x[:4]+'-'+x[4:6]+'-'+x[6:])

buy_pivot.to_csv('../data/sale_data.csv', index = False, encoding = 'CP949')

In [16]:
buy_pivot[buy_pivot.날짜=='2018-07-29'].sort_values(['일별 판매 합계량(개)'], ascending=False).head(20).소분류

76709                생수
76493         기초 화장용 크림
76776              커피음료
76812                 회
76672             돼지 곱창
76562                샴푸
76772             카페 푸드
76503        네일 메이크업 용품
76491        기초 화장용 에센스
76720                 쌀
76673                두유
76532    베이스 메이크업용 쿠션팩트
76585             클렌징 폼
76734               에이드
76572         스킨케어 마스크팩
76754            인스턴트커피
76783              탄산음료
76813               흰우유
76702               비타민
76668              닭가슴살
Name: 소분류, dtype: object

#### 5) 날씨 및 판매 데이터 병합

In [ ]:
sale_total_df = pd.read_csv('../data/sale_data.csv', encoding = 'CP949')
sale_total_df['날짜'] = pd.to_datetime(sale_total_df['날짜'])
sale_total_df.head()

,날짜,대분류,소분류,20대 여성 판매량(개),20대 남성 판매량(개),30대 여성 판매량(개),30대 남성 판매량(개),40대 여성 판매량(개),40대 남성 판매량(개),50대 여성 판매량(개),50대 남성 판매량(개),60대 여성 판매량(개),60대 남성 판매량(개),일별 판매 합계량(개)
0,2018-01-01,냉난방가전,가열식 가습기,1.0,1.0,0.0,0.0,3.0,2.0,0.0,0.0,0.0,0.0,7.0
1,2018-01-01,냉난방가전,공기정화 용품,10.0,2.0,35.0,18.0,14.0,24.0,2.0,12.0,2.0,1.0,120.0
2,2018-01-01,냉난방가전,공기청정기,1.0,2.0,28.0,26.0,16.0,18.0,6.0,4.0,2.0,3.0,106.0
3,2018-01-01,냉난방가전,냉풍기,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
4,2018-01-01,냉난방가전,돈풍기,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0


In [ ]:
weather_total_df.head()

,날짜,평균기온(°C),일교차(°C),일강수량(mm),평균 풍속(m/s),평균 상대습도(%),평균 전운량(1/10),오존(O3),미세먼지(PM10)
0,2018-01-01,0.066157,8.198700,0.000000,1.844893,41.267581,0.731871,0.015544,44.032069
1,2018-01-02,-0.000201,7.910404,0.000000,2.003189,47.229263,2.787500,0.013179,53.742579
2,2018-01-03,-2.328937,6.682618,0.000000,2.512411,39.916465,3.425790,0.017566,34.601134
3,2018-01-04,-2.419582,7.470117,0.000000,1.780529,44.077518,6.876999,0.011983,42.672343
4,2018-01-05,-0.734189,7.276539,0.114874,1.984833,50.195533,3.891405,0.011894,51.988526


In [ ]:
weather_sale_df = pd.merge(sale_total_df, weather_total_df, left_on = '날짜', right_on = '날짜', how = 'left')

weather_sale_df = weather_sale_df.reindex(['날짜', '대분류', '소분류',
                          '평균기온(°C)', '일교차(°C)', '일강수량(mm)', '평균 풍속(m/s)',
                           '평균 상대습도(%)','평균 전운량(1/10)', '오존(O3)', '미세먼지(PM10)',
                           '20대 여성 판매량(개)', '20대 남성 판매량(개)', '30대 여성 판매량(개)',
                           '30대 남성 판매량(개)', '40대 여성 판매량(개)', '40대 남성 판매량(개)',
                           '50대 여성 판매량(개)','50대 남성 판매량(개)', '60대 여성 판매량(개)',
                           '60대 남성 판매량(개)', '일별 판매 합계량(개)'], axis=1)

weather_sale_df.to_csv('../data/weather_sale_data.csv', encoding = 'cp949', index=False)

In [ ]:
weather_sale_df.head()

,날짜,대분류,소분류,평균기온(°C),일교차(°C),일강수량(mm),평균 풍속(m/s),평균 상대습도(%),평균 전운량(1/10),오존(O3),미세먼지(PM10),20대 여성 판매량(개),20대 남성 판매량(개),30대 여성 판매량(개),30대 남성 판매량(개),40대 여성 판매량(개),40대 남성 판매량(개),50대 여성 판매량(개),50대 남성 판매량(개),60대 여성 판매량(개),60대 남성 판매량(개),일별 판매 합계량(개)
0,2018-01-01,냉난방가전,가열식 가습기,0.066157,8.1987,0.0,1.844893,41.267581,0.731871,0.015544,44.032069,1.0,1.0,0.0,0.0,3.0,2.0,0.0,0.0,0.0,0.0,7.0
1,2018-01-01,냉난방가전,공기정화 용품,0.066157,8.1987,0.0,1.844893,41.267581,0.731871,0.015544,44.032069,10.0,2.0,35.0,18.0,14.0,24.0,2.0,12.0,2.0,1.0,120.0
2,2018-01-01,냉난방가전,공기청정기,0.066157,8.1987,0.0,1.844893,41.267581,0.731871,0.015544,44.032069,1.0,2.0,28.0,26.0,16.0,18.0,6.0,4.0,2.0,3.0,106.0
3,2018-01-01,냉난방가전,냉풍기,0.066157,8.1987,0.0,1.844893,41.267581,0.731871,0.015544,44.032069,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
4,2018-01-01,냉난방가전,돈풍기,0.066157,8.1987,0.0,1.844893,41.267581,0.731871,0.015544,44.032069,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0


##### 상관계수 계산

*   날씨와 소비자 카테고리 별 판매량 간의 상관계수를 계산
*   상관계수 기준을 0.3으로 두고 어떤 소비자 카테고리와도 상관계수가 0.3 이상으로 나오지 않은 품목은 제외
*   383가지 품목 중 최종 선택된 품목은 126가지







In [ ]:
sale_total_df.columns = ['날짜', '대분류', '소분류','20F', '20M', '30F', '30M', '40F', '40M', '50F', '50M', '60F', '60M', 'total' ]

In [ ]:
sale_pivot = pd.pivot_table(data = sale_total_df,
                            index = '날짜',
                            columns = ['소분류'],
                            values = sale_total_df.columns[3:],
                            aggfunc = 'sum')

In [ ]:
sale_pivot = sale_pivot.fillna(0).reset_index()
sale_pivot

날짜   20F                     ... total                              
소분류              가공란 가스온수기 가열식 가습기  가자미  ...  황토매트       회 휴대용 선풍기    흰우유     히터
0   2018-01-01  37.0   0.0     1.0  3.0  ...   4.0   847.0     1.0  600.0  119.0
1   2018-01-02  41.0   0.0     0.0  1.0  ...   6.0   974.0     4.0  804.0  287.0
2   2018-01-03  29.0   0.0     1.0  1.0  ...   7.0  1287.0     1.0  698.0  284.0
3   2018-01-04  24.0   0.0     1.0  0.0  ...   7.0  1424.0     3.0  602.0  242.0
4   2018-01-05  22.0   0.0     2.0  1.0  ...   7.0  1340.0     3.0  558.0  242.0
..         ...   ...   ...     ...  ...  ...   ...     ...     ...    ...    ...
725 2019-12-27  22.0   0.0     1.0  2.0  ...   3.0  1557.0     2.0  732.0  233.0
726 2019-12-28  20.0   0.0     0.0  0.0  ...   4.0  1696.0     5.0  668.0   79.0
727 2019-12-29  20.0   0.0     0.0  2.0  ...  11.0  1576.0     2.0  833.0  122.0
728 2019-12-30  35.0   0.0     0.0  2.0  ...  16.0  1636.0     1.0  915.0  183.0
729 2019-12-31  43.0   0.0     1.0  4.0  ...   8.0  2459.0     6.0  782.0  203.0

[730 rows x 4214 columns]

In [ ]:
sale_pivot_weather = pd.concat([sale_pivot, weather_total_df], axis=1).drop('날짜', axis = 1)

In [ ]:
sale_weather_corr = sale_pivot_weather.corr()

In [ ]:
sale_weather_corr

,"(20F, 가공란)","(20F, 가스온수기)","(20F, 가열식 가습기)","(20F, 가자미)","(20F, 갈비/찜/바비큐용 돈육)","(20F, 갈비용 우육)","(20F, 갈치)","(20F, 감/홍시)","(20F, 감귤/한라봉/오렌지)","(20F, 감마리놀렌산 영양제)","(20F, 감말랭이)","(20F, 감자)","(20F, 갓김치)","(20F, 건강즙)","(20F, 건강즙/녹용)","(20F, 건대추)","(20F, 건망고)","(20F, 건바나나)","(20F, 건어물 건새우)","(20F, 건어물 노가리)","(20F, 건어물 마른오징어)","(20F, 건어물 멸치)","(20F, 건어물 쥐포)","(20F, 건어물 진미채)","(20F, 건어물 황태)","(20F, 건자두)","(20F, 건포도)","(20F, 게장류)","(20F, 견과류)","(20F, 견과류 땅콩)","(20F, 견과류 마카다미아)","(20F, 견과류 밤)","(20F, 견과류 잣/은행)","(20F, 견과류 카카오닙스)","(20F, 견과류 캐슈넛)","(20F, 견과류 피스타치오)","(20F, 견과류 호두)","(20F, 계란)","(20F, 고등어)","(20F, 고추/피망/파프리카)",...,"(total, 허브차)","(total, 헛개/가시오가피)","(total, 헤어 브러쉬)","(total, 헤어매니큐어)","(total, 헤어무스)","(total, 헤어스타일링용 염색약)","(total, 헤어스타일링용 펌제)","(total, 헤어스타일링용 흑채)","(total, 헤어스프레이)","(total, 헤어에센스)","(total, 헤어왁스)","(total, 헤어젤)","(total, 헤어케어세트)","(total, 호박)","(total, 혼합견과)","(total, 혼합곡)","(total, 홍삼 간식)","(total, 홍삼 분말/환)","(total, 홍삼 음료)","(total, 홍삼/인삼 제품)","(total, 홍삼액/홍삼정)","(total, 홍삼절편/홍삼정과)","(total, 홍어)","(total, 홍차)","(total, 화장 비누)","(total, 화장 퍼프)","(total, 환자식)","(total, 황토매트)","(total, 회)","(total, 휴대용 선풍기)","(total, 흰우유)","(total, 히터)",평균기온(°C),일교차(°C),일강수량(mm),평균 풍속(m/s),평균 상대습도(%),평균 전운량(1/10),오존(O3),미세먼지(PM10)
"(20F, 가공란)",1.000000,-0.014066,-0.018737,0.023018,-0.164453,-0.085373,0.029257,-0.020502,0.022695,0.122114,0.011541,0.153902,0.094158,0.243106,0.179712,0.091233,0.099773,0.263370,0.046730,0.040534,0.199509,0.026031,0.126012,0.176295,0.025125,0.190822,0.118603,0.192991,0.228763,0.095333,0.120893,0.123017,0.048788,0.151578,0.147674,0.043460,0.032665,0.109886,0.162750,0.219331,...,0.311399,0.251562,0.222509,0.038635,0.207890,0.333548,0.249578,0.277318,0.405010,0.320217,0.236404,0.392094,0.148687,0.340696,0.088872,0.098125,0.287245,0.192988,0.099228,0.092595,0.185147,0.083998,0.075566,0.373031,0.357986,0.360805,0.347235,-0.039298,-0.029627,-0.010789,0.487582,-0.050489,-0.045599,0.145850,0.009865,0.000623,-0.076356,-0.000992,0.135446,0.191037
"(20F, 가스온수기)",-0.014066,1.000000,-0.024508,-0.049797,-0.043524,-0.004790,-0.049957,-0.045118,-0.038407,0.041371,0.017032,0.116020,0.033582,0.081450,0.106385,-0.021270,-0.010487,-0.011325,-0.017079,0.015190,0.082101,-0.062605,-0.010878,-0.008732,-0.023600,0.026622,-0.016177,-0.010420,-0.017377,-0.002894,-0.009652,-0.058097,0.007341,-0.028098,0.027926,-0.000766,-0.063340,0.030350,-0.022220,-0.012578,...,-0.030358,0.004577,0.007447,-0.008704,0.014810,0.099474,0.031542,-0.015322,0.047459,-0.015876,0.024289,0.029351,0.012450,0.002517,-0.006303,-0.005149,0.027560,0.053603,0.054817,0.037440,0.056496,0.075529,0.397852,0.014891,0.046786,0.009734,0.048310,-0.035960,-0.016645,-0.000640,-0.012555,-0.028598,0.020784,0.038653,-0.019653,-0.001625,-0.015741,0.033604,0.060898,-0.008657
"(20F, 가열식 가습기)",-0.018737,-0.024508,1.000000,0.026916,-0.000891,0.076857,0.013614,0.146229,0.443117,0.059550,0.208743,-0.085636,-0.043562,0.018322,0.018564,0.105249,-0.020022,-0.020885,0.009152,0.103870,0.088954,0.020483,-0.026272,0.122123,0.153603,-0.136130,-0.054413,0.223509,-0.018066,0.086693,-0.016476,0.179569,0.053250,-0.132700,-0.026616,0.019772,0.019381,0.029084,0.141862,0.054044,...,-0.001193,-0.031644,0.024635,-0.046725,0.023373,0.011841,0.073758,0.182625,0.041915,0.178489,-0.049187,0.019499,-0.073628,0.055761,-0.003997,-0.046818,-0.026285,-0.070653,-0.003817,-0.065242,-0.058880,-0.021237,0.103850,0.233209,-0.056927,0.087719,0.002000,0.366615,0.250950,-0.227698,0.132577,0.458990,-0.327298,-0.010880,-0.089118,-0.055586,-0.122830,-0.091858,-0.356634,0.048028
"(20F, 가자미)",0.023018,-0.049797,0.026916,1.000000,0.027103,0.076014,0.065904,0.053144,0.072855,0.105862,0.042640,0.018290,0.050207,0.044004,-0.004081,0.025618,0.048462,0.023157,-0.006945,0.080925,0.127983,0.094886,0.011430,0.066461,0.045665,-0.048340,-0.029389,0.055880,0.012793,0.022221,0.000502,0.106846,0.019290,-0.009628,-0.002984,0.025112,-0.057931,-0.036274,0.058656,0.077991,...,0.011740,0.085880,0.019251,-0.025425,0.069842,0.0

In [ ]:
new_sale_weather_corr = sale_weather_corr.iloc[:-8, -8:]

new_sale_weather_corr.to_csv('../data/new_sale_weather_corr.csv', encoding = 'CP949')

상관계수 기준을 0.3으로 두고 어떤 소비자 카테고리와도 상관계수가 0.3 이상으로 나오지 않은 품목은 제외**(엑셀로 처리)**

In [ ]:
# 선택한 상품 목록
selected_list = ['가열식 가습기','감/홍시','감귤/한라봉/오렌지','감말랭이','감자','건포도','견과류 밤','계란','고추/피망/파프리카','공기정화 용품','공기청정기','과일류','과채 음료/주스','굴 생물','기능성 아이케어 화장품','기초 화장용 로션','기초 화장용 미스트','기초 화장용 오일/앰플','기초 화장용 크림','기타 주스류','김치류','남성 로션','남성 메이크업','남성 선케어','남성 세트','남성 에센스','냉풍기','네일 메이크업 용품','다이어트보조식','다이어트용 헬스보충식품','대게/킹크랩','더치커피','데오드란트','돈풍기','둥굴레차','딸기/복분자/블루베리','딸기우유','라디에이터','레몬/자몽','멀티형 에어컨','메이크업 브러쉬','무김치','물김치','미숫가루/곡물가루','바나나/파인애플/망고','바디 보습제','바디 세트','바디 클렌져','바디케어용 땀패드','바디케어용 때비누','바디케어용 제모제','배/포도/과일즙','배추김치','베이스 메이크업용 쿠션팩트','베이스 메이크업용 파우더팩트','벽걸이 에어컨','벽걸이형 선풍기','복합식 가습기','뷰티 타투','비타민/화이바 음료','색조 메이크업 립밤','색조 메이크업 립스틱','색조 메이크업 마스카라','색조 메이크업 아이섀도우','생수','선로션','선스프레이','선케어용 선밤','선크림','스킨케어 코팩','스탠드형 냉온풍기','스탠드형 에어컨','시금치','아이스티','애프터선','어린이 음료','업소용 선풍기','에어워셔','에어컨 리모컨','에이드','오이/가지','옥수수','온수매트','온열매트','온풍기','유자차','유제품 음료','율무차','음용 식초','이동형 에어컨','이온음료','인스턴트커피','자연식 가습기','장어','전기온수기','전기장판','전통차','절임배추/김치속','제습기','조개','차/곡물 음료','참외/메론/수박','초음파식 가습기','카페 푸드','카페트매트','컨벡터','코코아/핫초코','키위/참다래','탁상/USB 선풍기','탄산수','탄산음료','태닝용 선크림','토마토','파/양파','풋크림','프로폴리스/로얄젤리','해초류 ','핸드크림','허브차','헤어에센스','호박','홍차','황토매트','회','휴대용 선풍기','히터']
print(selected_list)

['가열식 가습기', '감/홍시', '감귤/한라봉/오렌지', '감말랭이', '감자', '건포도', '견과류 밤', '계란', '고추/피망/파프리카', '공기정화 용품', '공기청정기', '과일류', '과채 음료/주스', '굴 생물', '기능성 아이케어 화장품', '기초 화장용 로션', '기초 화장용 미스트', '기초 화장용 오일/앰플', '기초 화장용 크림', '기타 주스류', '김치류', '남성 로션', '남성 메이크업', '남성 선케어', '남성 세트', '남성 에센스', '냉풍기', '네일 메이크업 용품', '다이어트보조식', '다이어트용 헬스보충식품', '대게/킹크랩', '더치커피', '데오드란트', '돈풍기', '둥굴레차', '딸기/복분자/블루베리', '딸기우유', '라디에이터', '레몬/자몽', '멀티형 에어컨', '메이크업 브러쉬', '무김치', '물김치', '미숫가루/곡물가루', '바나나/파인애플/망고', '바디 보습제', '바디 세트', '바디 클렌져', '바디케어용 땀패드', '바디케어용 때비누', '바디케어용 제모제', '배/포도/과일즙', '배추김치', '베이스 메이크업용 쿠션팩트', '베이스 메이크업용 파우더팩트', '벽걸이 에어컨', '벽걸이형 선풍기', '복합식 가습기', '뷰티 타투', '비타민/화이바 음료', '색조 메이크업 립밤', '색조 메이크업 립스틱', '색조 메이크업 마스카라', '색조 메이크업 아이섀도우', '생수', '선로션', '선스프레이', '선케어용 선밤', '선크림', '스킨케어 코팩', '스탠드형 냉온풍기', '스탠드형 에어컨', '시금치', '아이스티', '애프터선', '어린이 음료', '업소용 선풍기', '에어워셔', '에어컨 리모컨', '에이드', '오이/가지', '옥수수', '온수매트', '온열매트', '온풍기', '유자차', '유제품 음료', '율무차', '음용 식초', '이동형 에어컨', '이온음료', '인스턴트커피', '자연식 가습기', '장어', '전기온수기', '전기장판', '전통차', '절임

#### 6) SNS 데이터 전처리

In [ ]:
sns18_1 = pd.read_csv('../data/sns2018_1.csv')
sns18_2 = pd.read_csv('../data/sns2018_2.csv')
sns19_1 = pd.read_csv('../data/sns2019_1.csv')
sns19_2 = pd.read_csv('../data/sns2019_2.csv')

sns18_1.drop('Unnamed: 0', axis=1, inplace=True)
sns18_2.drop('Unnamed: 0', axis=1, inplace=True)
sns19_1.drop('Unnamed: 0', axis=1, inplace=True)
sns19_2.drop('Unnamed: 0', axis=1, inplace=True)

sns18_1.columns = ['date', 'big_cat', 'sm_cat', 'cnt']
sns18_2.columns = ['date', 'big_cat', 'sm_cat', 'cnt']
sns19_1.columns = ['date', 'big_cat', 'sm_cat', 'cnt']
sns19_2.columns = ['date', 'big_cat', 'sm_cat', 'cnt']

In [ ]:
sns_df = pd.concat([sns18_1, sns18_2, sns19_1, sns19_2], axis=0)
sns_df.date = sns_df.date.astype('str').map(lambda x:x[:4]+'-'+x[4:6]+'-'+x[6:])
sns_df.date = pd.to_datetime(sns_df.date)

In [ ]:
sns_df.head()

,date,big_cat,sm_cat,cnt
0,2018-01-01,뷰티,기능성 링클케어 화장품,12.154295
1,2018-01-01,뷰티,기능성 모공관리 화장품,36.000828
2,2018-01-01,뷰티,기능성 아이케어 화장품,0.895782
3,2018-01-01,뷰티,기능성 영양보습 화장품,14.868175
4,2018-01-01,뷰티,기능성 트러블케어 화장품,48.819391


In [ ]:
weather_sns_df = pd.merge(sns_df, weather_total_df,left_on='date', right_on='날짜', how='left')
weather_sns_df.drop('날짜', axis = 1, inplace = True)

In [ ]:
weather_sns_df = weather_sns_df.reindex(['date','big_cat', 'sm_cat','평균기온(°C)', '일교차(°C)',
                           '일강수량(mm)','평균 풍속(m/s)', '평균 상대습도(%)', '평균 전운량(1/10)',
                           '오존(O3)', '미세먼지(PM10)','cnt'], axis=1)

In [ ]:
weather_sns_df.columns = ['날짜', '대분류', '소분류','평균기온(°C)', '일교차(°C)','일강수량(mm)',
                   '평균 풍속(m/s)', '평균 상대습도(%)', '평균 전운량(1/10)','오존(O3)', '미세먼지(PM10)','SNS언급량' ]

In [ ]:
weather_sns_df.head()

,날짜,대분류,소분류,평균기온(°C),일교차(°C),일강수량(mm),평균 풍속(m/s),평균 상대습도(%),평균 전운량(1/10),오존(O3),미세먼지(PM10),SNS언급량
0,2018-01-01,뷰티,기능성 링클케어 화장품,0.066157,8.1987,0.0,1.844893,41.267581,0.731871,0.015544,44.032069,12.154295
1,2018-01-01,뷰티,기능성 모공관리 화장품,0.066157,8.1987,0.0,1.844893,41.267581,0.731871,0.015544,44.032069,36.000828
2,2018-01-01,뷰티,기능성 아이케어 화장품,0.066157,8.1987,0.0,1.844893,41.267581,0.731871,0.015544,44.032069,0.895782
3,2018-01-01,뷰티,기능성 영양보습 화장품,0.066157,8.1987,0.0,1.844893,41.267581,0.731871,0.015544,44.032069,14.868175
4,2018-01-01,뷰티,기능성 트러블케어 화장품,0.066157,8.1987,0.0,1.844893,41.267581,0.731871,0.015544,44.032069,48.819391


In [ ]:
weather_sns_df.to_csv('../data/weather_sns_data.csv', encoding = 'CP949', index = False)

#### 7) 머신러닝 및 딥러닝을 하기 위한 원핫인코딩


##### (1) 판매 데이터 필터링 및 원핫인코딩

In [ ]:
print(selected_list)

['가열식 가습기', '감/홍시', '감귤/한라봉/오렌지', '감말랭이', '감자', '건포도', '견과류 밤', '계란', '고추/피망/파프리카', '공기정화 용품', '공기청정기', '과일류', '과채 음료/주스', '굴 생물', '기능성 아이케어 화장품', '기초 화장용 로션', '기초 화장용 미스트', '기초 화장용 오일/앰플', '기초 화장용 크림', '기타 주스류', '김치류', '남성 로션', '남성 메이크업', '남성 선케어', '남성 세트', '남성 에센스', '냉풍기', '네일 메이크업 용품', '다이어트보조식', '다이어트용 헬스보충식품', '대게/킹크랩', '더치커피', '데오드란트', '돈풍기', '둥굴레차', '딸기/복분자/블루베리', '딸기우유', '라디에이터', '레몬/자몽', '멀티형 에어컨', '메이크업 브러쉬', '무김치', '물김치', '미숫가루/곡물가루', '바나나/파인애플/망고', '바디 보습제', '바디 세트', '바디 클렌져', '바디케어용 땀패드', '바디케어용 때비누', '바디케어용 제모제', '배/포도/과일즙', '배추김치', '베이스 메이크업용 쿠션팩트', '베이스 메이크업용 파우더팩트', '벽걸이 에어컨', '벽걸이형 선풍기', '복합식 가습기', '뷰티 타투', '비타민/화이바 음료', '색조 메이크업 립밤', '색조 메이크업 립스틱', '색조 메이크업 마스카라', '색조 메이크업 아이섀도우', '생수', '선로션', '선스프레이', '선케어용 선밤', '선크림', '스킨케어 코팩', '스탠드형 냉온풍기', '스탠드형 에어컨', '시금치', '아이스티', '애프터선', '어린이 음료', '업소용 선풍기', '에어워셔', '에어컨 리모컨', '에이드', '오이/가지', '옥수수', '온수매트', '온열매트', '온풍기', '유자차', '유제품 음료', '율무차', '음용 식초', '이동형 에어컨', '이온음료', '인스턴트커피', '자연식 가습기', '장어', '전기온수기', '전기장판', '전통차', '절임

In [ ]:
selected_index_sale = []
for i in selected_list:
    selected_index_sale.append(weather_sale_df['소분류'] == i)
selected_index_sale_array = np.array(selected_index_sale)

weather_sale_filtered_df = weather_sale_df[selected_index_sale_array.sum(axis = 0).astype('bool')]

In [ ]:
wea_sale_ohe = pd.get_dummies(weather_sale_filtered_df, columns = ['소분류'])

In [ ]:
wea_sale_ohe.head()

,날짜,대분류,평균기온(°C),일교차(°C),일강수량(mm),평균 풍속(m/s),평균 상대습도(%),평균 전운량(1/10),오존(O3),미세먼지(PM10),20대 여성 판매량(개),20대 남성 판매량(개),30대 여성 판매량(개),30대 남성 판매량(개),40대 여성 판매량(개),40대 남성 판매량(개),50대 여성 판매량(개),50대 남성 판매량(개),60대 여성 판매량(개),60대 남성 판매량(개),일별 판매 합계량(개),소분류_가열식 가습기,소분류_감/홍시,소분류_감귤/한라봉/오렌지,소분류_감말랭이,소분류_감자,소분류_건포도,소분류_견과류 밤,소분류_계란,소분류_고추/피망/파프리카,소분류_공기정화 용품,소분류_공기청정기,소분류_과일류,소분류_과채 음료/주스,소분류_굴 생물,소분류_기능성 아이케어 화장품,소분류_기초 화장용 로션,소분류_기초 화장용 미스트,소분류_기초 화장용 오일/앰플,소분류_기초 화장용 크림,...,소분류_유제품 음료,소분류_율무차,소분류_음용 식초,소분류_이동형 에어컨,소분류_이온음료,소분류_인스턴트커피,소분류_자연식 가습기,소분류_장어,소분류_전기온수기,소분류_전기장판,소분류_전통차,소분류_절임배추/김치속,소분류_제습기,소분류_조개,소분류_차/곡물 음료,소분류_참외/메론/수박,소분류_초음파식 가습기,소분류_카페 푸드,소분류_카페트매트,소분류_컨벡터,소분류_코코아/핫초코,소분류_키위/참다래,소분류_탁상/USB 선풍기,소분류_탄산수,소분류_탄산음료,소분류_태닝용 선크림,소분류_토마토,소분류_파/양파,소분류_풋크림,소분류_프로폴리스/로얄젤리,소분류_해초류,소분류_핸드크림,소분류_허브차,소분류_헤어에센스,소분류_호박,소분류_홍차,소분류_황토매트,소분류_회,소분류_휴대용 선풍기,소분류_히터
0,2018-01-01,냉난방가전,0.066157,8.1987,0.0,1.844893,41.267581,0.731871,0.015544,44.032069,1.0,1.0,0.0,0.0,3.0,2.0,0.0,0.0,0.0,0.0,7.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2018-01-01,냉난방가전,0.066157,8.1987,0.0,1.844893,41.267581,0.731871,0.015544,44.032069,10.0,2.0,35.0,18.0,14.0,24.0,2.0,12.0,2.0,1.0,120.0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2018-01-01,냉난방가전,0.066157,8.1987,0.0,1.844893,41.267581,0.731871,0.015544,44.032069,1.0,2.0,28.0,26.0,16.0,18.0,6.0,4.0,2.0,3.0,106.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2018-01-01,냉난방가전,0.066157,8.1987,0.0,1.844893,41.267581,0.731871,0.015544,44.032069,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2018-01-01,냉난방가전,0.066157,8.1987,0.0,1.844893,41.267581,0.731871,0.015544,44.032069,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
wea_sale_ohe = wea_sale_ohe.reindex(['날짜', '대분류']
                                  + wea_sale_ohe.loc[:,'소분류_가열식 가습기':].columns.tolist()
                                  + wea_sale_ohe.loc[:,'평균기온(°C)':'일별 판매 합계량(개)'].columns.tolist(),
                                  axis=1)

In [ ]:
wea_sale_ohe.to_csv('../data/weather_sale_ohe.csv', index = False)

##### (2) SNS 데이터 필터링 및 원핫인코딩

In [ ]:
weather_sns_df.head()

,날짜,대분류,소분류,평균기온(°C),일교차(°C),일강수량(mm),평균 풍속(m/s),평균 상대습도(%),평균 전운량(1/10),오존(O3),미세먼지(PM10),SNS언급량
0,2018-01-01,뷰티,기능성 링클케어 화장품,0.066157,8.1987,0.0,1.844893,41.267581,0.731871,0.015544,44.032069,12.154295
1,2018-01-01,뷰티,기능성 모공관리 화장품,0.066157,8.1987,0.0,1.844893,41.267581,0.731871,0.015544,44.032069,36.000828
2,2018-01-01,뷰티,기능성 아이케어 화장품,0.066157,8.1987,0.0,1.844893,41.267581,0.731871,0.015544,44.032069,0.895782
3,2018-01-01,뷰티,기능성 영양보습 화장품,0.066157,8.1987,0.0,1.844893,41.267581,0.731871,0.015544,44.032069,14.868175
4,2018-01-01,뷰티,기능성 트러블케어 화장품,0.066157,8.1987,0.0,1.844893,41.267581,0.731871,0.015544,44.032069,48.819391


In [ ]:
selected_index_sns = []
for i in selected_list:
    selected_index_sns.append(weather_sns_df['소분류'] == i)
selected_index_sns_array = np.array(selected_index_sns)

weather_sns_filtered_df = weather_sns_df[selected_index_sns_array.sum(axis = 0).astype('bool')]

In [ ]:
wea_sns_ohe = pd.get_dummies(weather_sns_filtered_df, columns = ['소분류'])
wea_sns_ohe = wea_sns_ohe.reindex(['날짜', '대분류']+wea_sns_ohe.columns.tolist()[11:]+wea_sns_ohe.columns.tolist()[2:11], axis=1)
wea_sns_ohe.index = range(0, len(wea_sns_ohe))

In [ ]:
wea_sns_ohe.head()

,날짜,대분류,소분류_가열식 가습기,소분류_감/홍시,소분류_감귤/한라봉/오렌지,소분류_감말랭이,소분류_감자,소분류_건포도,소분류_견과류 밤,소분류_계란,소분류_고추/피망/파프리카,소분류_공기정화 용품,소분류_공기청정기,소분류_과일류,소분류_과채 음료/주스,소분류_굴 생물,소분류_기능성 아이케어 화장품,소분류_기초 화장용 로션,소분류_기초 화장용 미스트,소분류_기초 화장용 오일/앰플,소분류_기초 화장용 크림,소분류_기타 주스류,소분류_김치류,소분류_남성 로션,소분류_남성 메이크업,소분류_남성 선케어,소분류_남성 세트,소분류_남성 에센스,소분류_냉풍기,소분류_네일 메이크업 용품,소분류_다이어트보조식,소분류_다이어트용 헬스보충식품,소분류_대게/킹크랩,소분류_더치커피,소분류_데오드란트,소분류_돈풍기,소분류_둥굴레차,소분류_딸기/복분자/블루베리,소분류_딸기우유,소분류_라디에이터,...,소분류_전기장판,소분류_전통차,소분류_절임배추/김치속,소분류_제습기,소분류_조개,소분류_차/곡물 음료,소분류_참외/메론/수박,소분류_초음파식 가습기,소분류_카페 푸드,소분류_카페트매트,소분류_컨벡터,소분류_코코아/핫초코,소분류_키위/참다래,소분류_탁상/USB 선풍기,소분류_탄산수,소분류_탄산음료,소분류_태닝용 선크림,소분류_토마토,소분류_파/양파,소분류_풋크림,소분류_프로폴리스/로얄젤리,소분류_해초류,소분류_핸드크림,소분류_허브차,소분류_헤어에센스,소분류_호박,소분류_홍차,소분류_황토매트,소분류_회,소분류_휴대용 선풍기,소분류_히터,평균기온(°C),일교차(°C),일강수량(mm),평균 풍속(m/s),평균 상대습도(%),평균 전운량(1/10),오존(O3),미세먼지(PM10),SNS언급량
0,2018-01-01,뷰티,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.066157,8.1987,0.0,1.844893,41.267581,0.731871,0.015544,44.032069,0.895782
1,2018-01-01,뷰티,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.066157,8.1987,0.0,1.844893,41.267581,0.731871,0.015544,44.032069,88.715916
2,2018-01-01,뷰티,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.066157,8.1987,0.0,1.844893,41.267581,0.731871,0.015544,44.032069,18.342442
3,2018-01-01,뷰티,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.066157,8.1987,0.0,1.844893,41.267581,0.731871,0.015544,44.032069,16.406682
4,2018-01-01,뷰티,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.066157,8.1987,0.0,1.844893,41.267581,0.731871,0.015544,44.032069,62.292207


In [ ]:
wea_sns_ohe.to_csv('../data/weather_sns_ohe.csv', index = False)